## Text to MySQL Queries
Building Q&A systems related to SQL database

In [11]:
import os
from dotenv import load_dotenv

load_dotenv("./../.env")

True

In [12]:
from langchain_community.utilities import SQLDatabase

In [13]:
username = "root"
password = ""
db_url = "@localhost"
db_name = "employee_db"

db = SQLDatabase.from_uri(f"mysql+pymysql://{username}:{db_url}/{db_name}")

In [14]:
print("Tables:", db.get_usable_table_names())
print(db.run(f"SELECT * FROM employees LIMIT 5"))

Tables: ['employees']
[(1, 'Ranna', 'Ryce', 'Female', '0000-00-00', '0000-00-00'), (2, 'Paten', 'Raise', 'Male', '0000-00-00', '0000-00-00'), (3, 'Maisey', 'Classen', 'Female', '0000-00-00', '0000-00-00'), (4, 'Nerita', 'Gilbard', 'Female', '0000-00-00', '0000-00-00'), (5, 'Irwin', 'Ciobutaru', 'Male', '0000-00-00', '0000-00-00')]


In [15]:
from langchain_ollama import ChatOllama 
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough 
from langchain_core.prompts import ChatPromptTemplate

from langchain_core.tools import tool

In [16]:
model = 'llama3.2'
base_url = 'http://localhost:11434'
ollama = ChatOllama(model=model, base_url=base_url)

In [17]:
from langchain.chains import create_sql_query_chain

In [18]:
sql_chain = create_sql_query_chain(ollama, db)
sql_chain.get_prompts()[0].pretty_print()

You are a MySQL expert. Given an input question, first create a syntactically correct MySQL query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most 5 results using the LIMIT clause as per MySQL. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in backticks (`) to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use CURDATE() function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result of the S

In [19]:
question = "How many employees are there?"
response = sql_chain.invoke({'question': question})
print(response)

Answer: There are 3 employees.

SQLQuery:

SELECT COUNT(*) as total_employees FROM employees;


The answer is wrong.

### Extract the correct query

In [20]:
from langchain_core.runnables import chain

import os
import sys

# Get the absolute path to the directory containing the script to import
# Replace 'path/to/your/other_folder' with the actual relative or absolute path
other_folder_path = os.path.abspath('./../scripts') 

# Add the folder to sys.path
sys.path.insert(0, other_folder_path) 

import llm

In [21]:
@chain 
def get_correct_sql_query(inputs):
  context = inputs['context']
  question = inputs['question']
  
  instruction = f"""
  Use above context to fetch the correct SQL query for following question
  {question}
  
  Do not enclose query in ```sql and do not write preamble and explanation.
  You MUST return only single SQL query.
  """
  
  response = llm.ask_llm(context=context, question=instruction)
  return response
  

In [22]:
response = get_correct_sql_query.invoke({'context': response, 'question': question})
print(response)

SELECT COUNT(*) as total_employees FROM employees


In [23]:
db.run(response)

'[(1000,)]'

### Final query chain

In [24]:
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool

In [25]:
ollama = ChatOllama(model=model, base_url=base_url)

sql_query = create_sql_query_chain(ollama, db)
execute_query = QuerySQLDataBaseTool(db=db)

final_chain = (
  {'context': sql_query, "question": RunnablePassthrough()} 
  | get_correct_sql_query 
  | execute_query
)

In [26]:
response = final_chain.invoke({'question':  "How many employees are there?"})
print(response)

[(1000,)]


### Use agent instead of chain

In [27]:
from langchain_community.agent_toolkits import SQLDatabaseToolkit

In [28]:
toolkit = SQLDatabaseToolkit(db=db, llm=ollama)
tools = toolkit.get_tools()
tools

[QuerySQLDataBaseTool(description="Input to this tool is a detailed and correct SQL query, output is a result from the database. If the query is not correct, an error message will be returned. If an error is returned, rewrite the query, check the query, and try again. If you encounter an issue with Unknown column 'xxxx' in 'field list', use sql_db_schema to query the correct table fields.", db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x0000021886E2B6D0>),
 InfoSQLDatabaseTool(description='Input to this tool is a comma-separated list of tables, output is the schema and sample rows for those tables. Be sure that the tables actually exist by calling sql_db_list_tables first! Example Input: table1, table2, table3', db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x0000021886E2B6D0>),
 ListSQLDatabaseTool(db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x0000021886E2B6D0>),
 QuerySQLCheckerTool(description='Use this tool to 

In [29]:
from langchain_core.messages import SystemMessage

SQL_PREFIX = """
You are an agent designed to interact with a SQL database.
Given an input question, create a syntactically correct SQLite query to run, then look at the results of the query and return the answer.
Unless the user specifies a specific number of examples they wish to obtain, always limit your query to at most 5 results.
You can order the results by a relevant column to return the most interesting examples in the database.
Never query for all the columns from a specific table, only ask for the relevant columns given the question.
You have access to tools for interacting with the database.
Only use the below tools. Only use the information returned by the below tools to construct your final answer.
You MUST double check your query before executing it. If you get an error while executing a query, rewrite the query and try again.

DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the database.

To start you should ALWAYS look at the tables in the database to see what you can query.
Do NOT skip this step.
Then you should query the schema of the most relevant tables."""

system_message = SystemMessage(content=SQL_PREFIX)

In [30]:
from langchain_core.messages import HumanMessage
from langgraph.prebuilt import create_react_agent

In [31]:
ollama = ChatOllama(model=model, base_url=base_url)
agent_executor = create_react_agent(ollama, tools, state_modifier=system_message, debug=True)

In [32]:
question = "How many employees are there?"

agent_executor.invoke({"messages": [HumanMessage(content=question)]})

[-1:checkpoint] State at the end of step -1:
{'messages': []}
[0:tasks] Starting step 0 with 1 task:
- __start__ -> {'messages': [HumanMessage(content='How many employees are there?', additional_kwargs={}, response_metadata={})]}
[0:writes] Finished step 0 with writes to 1 channel:
- messages -> [HumanMessage(content='How many employees are there?', additional_kwargs={}, response_metadata={})]
[0:checkpoint] State at the end of step 0:
{'messages': [HumanMessage(content='How many employees are there?', additional_kwargs={}, response_metadata={}, id='cb82c284-4b27-4aba-90b0-6e83b3463f1c')]}
[1:tasks] Starting step 1 with 1 task:
- agent -> {'is_last_step': False,
 'messages': [HumanMessage(content='How many employees are there?', additional_kwargs={}, response_metadata={}, id='cb82c284-4b27-4aba-90b0-6e83b3463f1c')],
 'remaining_steps': 24}
[1:writes] Finished step 1 with writes to 1 channel:
- messages -> [AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'llama3.

{'messages': [HumanMessage(content='How many employees are there?', additional_kwargs={}, response_metadata={}, id='cb82c284-4b27-4aba-90b0-6e83b3463f1c'),
  AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'llama3.2', 'created_at': '2025-06-30T08:45:53.971265Z', 'done': True, 'done_reason': 'stop', 'total_duration': 19733043200, 'load_duration': 13013169000, 'prompt_eval_count': 716, 'prompt_eval_duration': 3995492500, 'eval_count': 19, 'eval_duration': 2716414000, 'message': Message(role='assistant', content='', images=None, tool_calls=[ToolCall(function=Function(name='sql_db_query', arguments={'query': 'SELECT COUNT(*) FROM employees'}))])}, id='run-2e1528a4-60b9-4bde-9f87-657fb468d4be-0', tool_calls=[{'name': 'sql_db_query', 'args': {'query': 'SELECT COUNT(*) FROM employees'}, 'id': 'fb269155-b5ba-403d-82a6-aadc497998b7', 'type': 'tool_call'}], usage_metadata={'input_tokens': 716, 'output_tokens': 19, 'total_tokens': 735}),
  ToolMessage(content='[(1000,)]', 